# MODELO PREDICTOR DE REPROBACIÓN CON REDES NEURONALES UTILANDO KERAS

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
import keras_tuner as kt
from keras_tuner import HyperModel, Objective
from keras_tuner.tuners import BayesianOptimization, Hyperband
from tensorflow.keras import backend as K
from numpy import loadtxt
import pandas as pd

In [4]:
print('tensorflow: %s' % tf.__version__)
print('keras: %s' % keras.__version__)

tensorflow: 2.6.1
keras: 2.6.0


# FUENTES DE CONSULTA

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

https://keras.io/guides/keras_tuner/getting_started/

https://www.kaggle.com/pratsbhatt/keras-tuner-with-bayesian-optimization

## importar datos de entrenamiento

In [5]:
# CONJUNTO DE ENTRENAMIENTO
train= pd.read_csv("predictores_train_VIF.csv")
train.columns

Index(['tasa_rep_carga', 'ceneval_analitico', 'ceneval_matematico',
       'ceneval_lengua', 'ceneval_esp', 'tasa_aprob_per_prev', 'asigMuchas',
       'carrera_IA', 'carrera_IDeIO', 'carrera_ILyCS', 'carrera_II',
       'carrera_IE', 'carrera_NI', 'carrera_TS', 'semestre_Oto�o',
       'complejidad_carga5', 'a�o_encurso', 'carga_aprobada',
       'total_recursando', 'practicante', 'situacion_Condicionado',
       'situacion_Irregular'],
      dtype='object')

In [7]:
x1 = train.loc[:, train.columns != "carga_aprobada"]

array([[6.79137307e+00, 1.03000000e+03, 9.40000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.20736058e+01, 1.03000000e+03, 9.40000000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [9.72222222e+00, 1.03000000e+03, 9.40000000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.24444444e+01, 1.06000000e+03, 1.11000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.19396669e+00, 1.06000000e+03, 1.03600000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.47581993e+00, 1.06000000e+03, 1.03600000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [8]:
# ETIQUETAS DEL CONJUNTO DE ENTRENAMIENTO
y1 = train.loc[:, train.columns == "carga_aprobada"]

array([0, 0, 1, ..., 1, 1, 1])

## se importa el conjunto de datos para la validación

In [9]:
# CONJUNTO DE VALIDACION
test= pd.read_csv("test_final.csv")
test.columns

Index(['tasa_rep_carga', 'ceneval_analitico', 'ceneval_matematico',
       'ceneval_lengua', 'ceneval_esp', 'tasa_aprob_per_prev', 'asigMuchas',
       'carrera_IA', 'carrera_IDeIO', 'carrera_ILyCS', 'carrera_II',
       'carrera_IE', 'carrera_NI', 'carrera_TS', 'semestre_Otoño',
       'complejidad_carga5', 'año_encurso', 'carga_aprobada',
       'total_recursando', 'practicante', 'situacion_Condicionado',
       'situacion_Irregular'],
      dtype='object')

## x2 y y2 son los predictores y etiquetas del conjunto de validacion respectivamente
## para uso del tuner estos se dividen a la mitad resultando en x2 y x3, asi como y2 y y3.
## los nuevos x2 y y2 se utilizan para la validacion durante el proceso de keras tuner
## x3 y y3 se utilizan para una última validación del mejor modelo resultante, importante saber que estos datos no jugaron ningun papel durante el entrenamiento

In [10]:
# PREDICTORES VALIDACION
x2 = test.loc[:, test.columns != "carga_aprobada"]

In [11]:
len(x2)/2

6070.0

In [12]:
# divide x2 in 2 halves, x2 & x3
x3 = x2.iloc[6070: , :]

In [13]:
x2 = x2.iloc[:6070 , :]

In [14]:
# ETIQUETAS VALIDACION
y2 = test.loc[:, test.columns == "carga_aprobada"]

In [15]:
y3 = y2.iloc[6070: , :] #y3 is og y2 second half

In [16]:
y2 = y2.iloc[:6070 , :]

# KERAS_TUNER

## el keras tuner itera diferentes versiones de modelos variando hiperparametros en cada vuelta
## va almacenando info de cada modelo la cual se puede consultar al final

In [17]:
# F-1 SE DEFINE FUNCION PARA USARLA COMO METRICA DURANTE EL ENTRENAMIENTO
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

## primero debe definirse un hipermodelo, como un prototipo donde se definen caracterisitcas del modelo:
## número de capas, nodos, metodos de activacion, optimizador, metricas, etc.

In [18]:
# Create the keras tuner model.
class MyHyperModel(HyperModel):
    
    def build(self, hp):
        model = Sequential()
        model.add(layers.InputLayer( input_shape=(21,) )) # PRIMERA CAPA CON UN NODO POR CADA ATRIBUTO        
        for i in range(hp.Int("num_layers", 1, 3)): # PARA NUMERO DE CAPAS ENTRE 1 Y 2
            model.add(
                layers.Dense(
                    units=hp.Int("units_" + str(i), min_value=2, max_value=42, step=2), # AGREGARA CAPAS CON NODOS ENTRE 2 Y 42
                    activation="relu",
                )
            )
        
        model.add(layers.Dense(1, activation="sigmoid")) # CAPA FINAL
        
        model.compile(
            optimizer=hp.Choice('optimizer', values= ['Adam', 'Adadelta', 'Adamax']),
            loss='binary_crossentropy',
            metrics=[f1, 'accuracy','AUC','Precision','Recall']) # METRICA F1 CREADA
        
        return model

## se inicializa un objeto del hipermodelo para utilizarlo en el objeto tuner
## el tuner es el iterador como tal y se definen otros parametros
## existen varios tipos de tuner, en este caso el objetivo o metrica principal es el f1

In [19]:
hypermodel = MyHyperModel()

# TUNER PARA AFINAR HIPERPARAMETROS
# TUNER HYPERBAND, MAS EFICIENTE COMPUTACIONALMENTE

tuner = Hyperband(
    hypermodel,
    objective=Objective('val_f1', direction="max"), # OBJETIVO: MAXIMIZAR F1
    max_epochs=30,
    factor=3,
    hyperband_iterations=2,
    seed=None,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    directory="newdir20" # CAMBIAR NOMBRE SI BUSQUEDA ARROJA: INFO:tensorflow:Oracle triggered exit
)

In [20]:
tuner.search_space_summary() # IMPRIMIR CARACTERISTICAS DEL TUNER

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 42, 'step': 2, 'sampling': None}
optimizer (Choice)
{'default': 'Adam', 'conditions': [], 'values': ['Adam', 'Adadelta', 'Adamax'], 'ordered': False}


## despues se ejecuta la busqueda, siempre señalando el mejor resultado encontrado
## hasta el momento se ejecuta un numero de pruebas indeterminado y se detiene 

In [21]:
#model2.fit(x1.to_numpy(), y1.carga_aprobada.to_numpy(), epochs=1000, batch_size=1)
tuner.search(x1.to_numpy(),y1.carga_aprobada.to_numpy(),
             epochs=10, validation_data=(x2.to_numpy(),y2.carga_aprobada.to_numpy()))

Trial 180 Complete [00h 00m 20s]
val_f1: 0.7708594799041748

Best val_f1 So Far: 0.774045467376709
Total elapsed time: 00h 17m 46s
INFO:tensorflow:Oracle triggered exit


## resumen de los resultados

In [22]:
tuner.results_summary()

Results summary
Results in newdir20/untitled_project
Showing 10 best trials
Objective(name='val_f1', direction='max')
Trial summary
Hyperparameters:
num_layers: 1
units_0: 24
optimizer: Adamax
units_1: 12
units_2: 18
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: f2780735a07ffebb5d1cd568ea87d22e
Score: 0.774045467376709
Trial summary
Hyperparameters:
num_layers: 2
units_0: 20
optimizer: Adamax
units_1: 12
units_2: 16
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.7734240889549255
Trial summary
Hyperparameters:
num_layers: 2
units_0: 12
optimizer: Adam
units_1: 4
units_2: 28
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7732095718383789
Trial summary
Hyperparameters:
num_layers: 2
units_0: 30
optimizer: Adam
units_1: 24
units_2: 36
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0a0438d86f6396074a90e02b17aaa9c7
Score: 0.7729024887084961
T

In [23]:
tuner.get_best_models(num_models=3) # apuntadores
# evaluate the keras model
# F,loss, accuracy, AUC, Precision, Recall = model2.evaluate(x2.to_numpy(), y2.carga_aprobada.to_numpy())

## el mejor modelo se puede consultar y volver a evaluar

## se evalua con respecto a un tercer conjunto de la muestra no perteneciente al entrenamiento o a la validación

In [24]:
# EVALUA EL MEJOR MODELO Y CALCULA LAS DEMAS METRICAS

#best_model = tuner_hb.get_best_models(num_models=1)[0]
#best_model.evaluate(x_test_scaled, y_test)
tuner.get_best_models(num_models=1)[0].evaluate( x3.to_numpy(),y3.carga_aprobada.to_numpy() )

190/190 [==============================] - 1s 1ms/step - loss: 0.5920 - f1: 0.7622 - accuracy: 0.7168 - auc: 0.7655 - precision: 0.7209 - recall: 0.8783


[0.592042863368988,
 0.7621918320655823,
 0.7168039679527283,
 0.765464186668396,
 0.7208994626998901,
 0.8783239126205444]

## despues de varias pruebas el f1 score mas alto obtenido ha sido de 0.77